In [15]:
import os
import sys
import math

import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import rotate, shift
from scipy.ndimage.morphology import binary_erosion

from skimage.measure import block_reduce

from skimage.transform import hough_line, hough_line_peaks


In [16]:
target_dir = "augmented_images_demo /"
max_width = 250
max_height = 1200


In [17]:
def binarize(img, thres):
    res = np.sum(img,axis=2) > thres
    return res.astype(int)

def downsample(img):
    res = block_reduce(img, block_size=(21, 21), func=np.mean)
    return res

def erode(img):
    res = binary_erosion(img, np.ones((3,3), dtype=np.int))
    return res

def find_angle(img):
    img = binarize(img, 2)
    img = downsample(img)
    img = erode(img)
    h_space, angles, dist = hough_line(img)
    h_space, angles, dist = hough_line_peaks(h_space, angles, dist, num_peaks=1)
    return math.degrees(angles[0])

def augment_rotation(img):
    """
    Finds out what the asparagus angle is, zeroes image by rotating it to 0°, then creates 
    three +/- 5° rotations of it, with noise added to the degrees by which the image is rotated
    """
    rotations = np.array([-1, 0, 1]) * 5 + np.random.normal(0,1.5) # TODO fix this, there is a more elegant way
    angle = find_angle(img)
    base = rotate(img, -1*angle, reshape=False, mode="constant")
    return [rotate(base, x, reshape=False, mode="constant") for x in rotations]

def augment_mirroring(img):
    return [np.fliplr(img)]

def augment_translation(img, num=5):
    
    def deviation(x):
        return np.random.choice(np.arange(-x,x))
    
    return [shift(img, [deviation(40), deviation(30), 0]) for _ in range(num)]


In [18]:
def augment_images(target_dir):
    """
    Expects a directory full of preprocessed jpg-files of single asparagus pieces.
    Applies all augmentation functions to all files in that folder and stores resulting images with
    appropriate prefix and ID to create unique names.
    """
    # storing all functions with respective args and 3-letter-prefixes (that's important to facilitate processing)
    func_args = [(augment_translation, 5, "tra"), (augment_rotation, None, "rot"), (augment_mirroring, None, "mir")]
    
    for func, args, prefix in func_args:
        
        # recollect all images
        images = sorted([f for f in os.listdir(target_dir) if os.path.isfile(os.path.join(target_dir, f)) and not f[0] == "." and f[-4:] == ".jpg"])
        
        for file in images:
            
            img = plt.imread(os.path.join(target_dir, file))
            
            if( not args is None):
                transformed = func(img, args)
            else:
                transformed = func(img)
                
            for idx, trans in enumerate(transformed):
                plt.imsave(os.path.join(target_dir,prefix+str(idx)+"_"+file),trans)
        

In [20]:
augment_images(target_dir)

KeyboardInterrupt: 